#  Aprendizaje Automático

# Trabajo Práctico 1 - Detección de Spam

# Estudiantes: Juan Manuel Perez, Mariela Rajngewerc, Tomás Freilij





#  Ayudas para nosotros

# https://sourceforge.net/p/jupiter/wiki/markdown_syntax/#md_ex_text


 PREGUNTA: ¿Cómo debe ser el informe?

RESPUESTA: Recomendamos armar el informe con las siguientes secciones:

    Extracción de atributos: Describir en castellano los atributos extraidos de los mails, en forma concisa.
    Modelos: Listar los algoritmos de aprendizaje elegidos para experimentar. Describir cualquier decisión que hayan tomado (p.ej., elección de hiperparámetros).
    Reducción de dimensionalidad: Describir brevemente las técnicas empleadas.
    Resultados: Describir los resultados conseguidos por los distintos modelos y conjuntos de atributos considerados. Preferentemente, resumir los resultados en tablas/figuras. Mencionar los tiempos de ejecución aproximados de cada técnica.
    Discusión: Analizar los resultados, buscando responder cuestiones como, por ejemplo: ¿cuáles son los atributos encontrados con mayor poder predictivo?, ¿cuán sensibles fueron los algoritmos a las técnicas de reducción de dimensionalidad consideradas?, ¿resultó clara la elección del algoritmo para la competencia, o hubo que poner en la balanza distintos factores?

La longitud sugerida del informe es de entre 3 y 5 páginas de texto (sin contar tablas o figuras). Además pueden incluirse tablas y figuras, pero siempre deben ser referenciadas y explicadas en el texto.

Si se tomaron ideas de la literatura (papers, libros, blogs, wikipedia o lo que sea), citar claramente las fuentes (autor, título, tipo de publicación, año de publicación, URL si corresponde, etc.).

No incluir código. Si es necesario describir un algoritmo, hacerlo en pseudocódigo

# Resumen del trabajo

El trabajo tuvo como punto central poder probar los distintos clasificadores pedidos por la cátedra, con las respectivas mejoras que se les pueda encontrar a cada uno. Las implementaciones usadas fueron las provistas por Scikit-Learn.

En términos generales, el esquema fue correr los clasificadores obteniendo distintas hipótesis según los hiperparámetros que podíamos modificar.

Luego se intentó encontrar la mejor opción. Para esto se usaron dos implementaciones de la librería, GridSearch y RandomizedSearch. La primera obtiene la mejor haciendo una búsqueda exhaustiva, pero resultó ser bastante costosa en términos de tiempos de ejecución. Por esto último optamos por RandomizedSearch, que si bien no garantiza encontrar la óptima, provee un buen trade-off entre tiempos y resultados.



# Extracción de Atributos

Los atributos principales que se extrajeron de los mails fueron:

- 

# Validación utilizada

In [ ]:
Utilizamos 10 fold cross-validation implementada por scikit learn.

# Decision Trees

# Los hiperparámetros que nos parecieron interesantes, provistos por la librería fueron.

** Criterio de partición ('criterion')** 

Establece la función que mide la calidad de una partición de los nodos del árbol. 
Existen dos provistos por la librería, "Gini" y "Entropía". El último responde al Information Gain visto en la materia.

**Profundidad máxima ('max_depth')**

Permite hacer una poda sobre el árbol. 

**Splitter**

Tiene dos opciones, best y random. Establece si se utiliza el mejor criterio de partición o usa un mecanismo aleatorio para buscarlo. Como ya dijimos antes, la calidad de la partición estará dada por el hiperparámetro 'criterion'

**Subconjunto de valores de un atributo('max_features')**

Establece el máximo de valores de un atributo que se tiene en cuenta. Visto más gráficamente, establece el ancho del árbol.
Se puede establecer un número fijo, o un porcentaje, entre otras posibilidades.

**Criterio de corte para un nodo('min_samples_split')**

La mínima cantidad de muestras necesarias para que se justifique la partición en un nodo. 


** La mejor combinación de hiperparámetros fue **

'min_samples_split' = 82 

'splitter' = 'best' 

'criterion' = 'entropy' 

'max_depth'= 18 

'max_features' = 0.90000000000000013

El valor fue 0.987803482259 con el scoring roc_auc.

 
# VER QUE ONDA CON EL ROC AUC SCORE QUE VARIA CON EL ANTERIOR MOSTRADO 


| precision_score | accuracy_score   |    f1_score 	| recall_score |	roc_auc_score |
|-----------------|------------------|--------------|--------------|------------------|
| 0.956531        | 0.957444         |    0.957487 	|  0.958444    |     0.957444     |



 	   	

 	             	    


# Naive Bayes
   

Dada la naturaleza del clasificador, hay que tener también en cuenta la distribución de las muestras. En ese sentido, la librería provee tres clasificadores de Naive Bayes, usando una distribución Gaussiana, una de Bernoulli y una Multinomial.

Las primeras pruebas arrojaron resultados nulos con la multinomial y la de bernoulli. Cuando hablamos de resultados nulos nos referimos a que, con pruebas varias que hacíamos, la clasificación era completamente errada.

Dado que la Gaussiana no arrojaba ese tipo de resultado, decidimos trabajar con esa propuesta.

Ahora bien, cuando graficábamos la distribución de algunos atributos en los mails veíamos que se parecía más a una Exponencial que a una Gaussiana. Además, los gráficos perdían proporcionalidad y no se podían entender. Al intentar aplicarle logaritmo a los datos para darle otra escala vimos que tomaban una forma de campa de gauss.

Como además aplicar el logaritmo preserva las desigualdades, nos parecía razonable usar esta conversión previa para trabajar con el clasificador.

Este clasificador no tiene parámetros así que no tuvimos que hacer pruebas variadas en relación a modificación de hiperparámetros.

Los resultados obtenidos sin previa normalización usando logaritmo fueron:

 |	precision_score  | accuracy_score  |	f1_score  |	recall_score  |	roc_auc_score | 
 |-------------------|-----------------|--------------|---------------|---------------|  
 |	0.504766 	     |0.509222 	       |    0.665556  |	0.976667 	  | 0.509222      |  
 
 Con normalización fueron:
 
| precision_score |	accuracy_score | f1_score |	recall_score | roc_auc_score |
|-----------------|----------------|----------|--------------|---------------|
| 1.0 	          | 0.500778 	   | 0.003106 |	0.001556 	 |  0.500778     |            
 

# AGREGAR LO DE BERNOULLI QUE APARECIÓ DESPUÉS

# KNN

# SVM

# RANDOM FOREST